# Clustering

지도 학습은 정답이 있는 데이터를 활용하여 학습을 하는 방법이다. 분류(Classification), 회귀(Regression) 문제가 있다.\
비지도 학습은 정답이 없는 데이터를 활용하여 결과를 예측하는 방법이다. 군집(Clustering), 이상탐지(Outier detection)가 있다.\
강화 학습은 데이터도, 정답도 정해지지 않았으며 행동에 대한 보상으로 학습한다.

K-Means는 군집화에서 가장 일반적인 알고리즘으로 군집 중심이라는 임의의 지점을 선택하여 해당 중심점에 가까운 포인트를 선택하는 군집화 방법이다.\
선택된 포인트들의 무게중심점을 새로운 군집중심으로 다시 탐색하여 군집중심을 찾아간다.\
거리기반 알고리즘으로 속성의 개수가 매우 많을 경우 군집화의 정확도가 떨어진다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs

In [ ]:
iris = load_iris()
cols = [each[:-5] for each in iris.feature_names]
iris_pd = pd.DataFrame(iris.data, columns=cols)
iris_pd.head()

In [ ]:
feature = iris_pd[['petal length', 'petal width']]
feature.head()

In [ ]:
# n_clusters는 군집 중심점의 수다.

model = KMeans(n_clusters=3)
model.fit(feature)

In [ ]:
model.labels_
model.cluster_centers_

In [ ]:
predict = pd.DataFrame(model.predict(feature), columns=['cluster'])
feature = pd.concat([feature, predict], axis=1)
feature.head()

In [ ]:
centers = pd.DataFrame(model.cluster_centers_, columns=['petal length', 'petal width'])
center_x = centers['petal length']
center_y = centers['petal width']
plt.figure(figsize=(12,8))
plt.scatter(feature['petal length'], feature['petal width'], c=feature['cluster'], alpha=0.5, cmap='Accent')
plt.scatter(center_x, center_y, s=50, marker='D', c='r')
plt.show()

In [ ]:
x, y = make_blobs(n_samples=200, n_features=2, centers=3, cluster_std=0.8, random_state=0)
print(x.shape, y.shape)
unique, counts = np.unique(y, return_counts=True)
print(unique, counts)

In [ ]:
cluster_df = pd.DataFrame(data=x, columns=['ftr1', 'ftr2'])
cluster_df['target'] = y
cluster_df.head()

In [ ]:
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=200, random_state=50)
cluster_labels = kmeans.fit_predict(x)
cluster_df['kmeans_label'] = cluster_labels

In [ ]:
centers = kmeans.cluster_centers_
unique_labels = np.unique(cluster_labels)
markers=['o','s','^','P','D','H','x']

for label in unique_labels :
    label_cluster = cluster_df[cluster_df['kmeans_label']==label]
    center_x_y = centers[label]
    plt.scatter(x=label_cluster['ftr1'],y=label_cluster['ftr2'], edgecolor='k', marker=markers[label])
    plt.scatter(x=center_x_y[0], y=center_x_y[1], s=200, color='white', alpha=0.9, edgecolor='k', marker=markers[label])
    plt.scatter(x=center_x_y[0], y=center_x_y[1], s=70, edgecolor='k', marker='$%d$'%label)


In [ ]:
print(cluster_df.groupby('target')['kmeans_label'].value_counts())

## 군집 결과의 평가
분류기는 평가 기준을 갖고있지만, 군집은 그렇지 않다.\
군집 결괄르 평가하기 위해서 실루엣 분석을 많이 활용한다.\
실루엣 분석은 각 군집간의 거리가 얼마나 효율적으로 분리되어 있는지 나타낸다.\
다른 군집과 떨어져 있고, 동일 군집간의 밀집도를 확인한다.\
군집화가 잘 되어있을 수록 개별 군집은 비슷한 여유공간을 갖는다.

In [ ]:
iris = load_iris()
feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
iris_df = pd.DataFrame(data=iris.data, columns=feature_names)
kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=300, random_state=0).fit(iris_df)

In [ ]:
iris_df['cluster'] = kmeans.labels_
iris_df.head()

In [ ]:
from sklearn.metrics import silhouette_samples, silhouette_score
avg_value = silhouette_score(iris.data, iris_df['cluster'])
score_values = silhouette_samples(iris.data, iris_df['cluster'])
print('avg_value', avg_value)
print('silhouette_samples() return 값의 shape ', score_values.shape)

In [ ]:
def visualize_silhouette(cluster_lists, X_features): 

    from sklearn.datasets import make_blobs
    from sklearn.cluster import KMeans
    from sklearn.metrics import silhouette_samples, silhouette_score

    import matplotlib.pyplot as plt
    import matplotlib.cm as cm
    import math

    # 입력값으로 클러스터링 갯수들을 리스트로 받아서, 각 갯수별로 클러스터링을 적용하고 실루엣 개수를 구함
    n_cols = len(cluster_lists)

    # plt.subplots()으로 리스트에 기재된 클러스터링 수만큼의 sub figures를 가지는 axs 생성 
    fig, axs = plt.subplots(figsize=(4*n_cols, 4), nrows=1, ncols=n_cols)

    # 리스트에 기재된 클러스터링 갯수들을 차례로 iteration 수행하면서 실루엣 개수 시각화
    for ind, n_cluster in enumerate(cluster_lists):

        # KMeans 클러스터링 수행하고, 실루엣 스코어와 개별 데이터의 실루엣 값 계산. 
        clusterer = KMeans(n_clusters = n_cluster, max_iter=500, random_state=0)
        cluster_labels = clusterer.fit_predict(X_features)

        sil_avg = silhouette_score(X_features, cluster_labels)
        sil_values = silhouette_samples(X_features, cluster_labels)

        y_lower = 10
        axs[ind].set_title('Number of Cluster : '+ str(n_cluster)+'\n' \
                           'Silhouette Score :' + str(round(sil_avg,3)) )
        axs[ind].set_xlabel("The silhouette coefficient values")
        axs[ind].set_ylabel("Cluster label")
        axs[ind].set_xlim([-0.1, 1])
        axs[ind].set_ylim([0, len(X_features) + (n_cluster + 1) * 10])
        axs[ind].set_yticks([])  # Clear the yaxis labels / ticks
        axs[ind].set_xticks([0, 0.2, 0.4, 0.6, 0.8, 1])

        # 클러스터링 갯수별로 fill_betweenx( )형태의 막대 그래프 표현. 
        for i in range(n_cluster):
            ith_cluster_sil_values = sil_values[cluster_labels==i]
            ith_cluster_sil_values.sort()

            size_cluster_i = ith_cluster_sil_values.shape[0]
            y_upper = y_lower + size_cluster_i

            color = cm.nipy_spectral(float(i) / n_cluster)
            axs[ind].fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_sil_values, \
                                facecolor=color, edgecolor=color, alpha=0.7)
            axs[ind].text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
            y_lower = y_upper + 10

        axs[ind].axvline(x=sil_avg, color="red", linestyle="--")

In [ ]:
visualize_silhouette([2,3,4,], iris.data)